## Selección de modelos

En el aprendizaje automático, generalmente seleccionamos nuestro modelo final después de evaluar varios modelos candidatos. Este proceso se llama *selección de modelo*. A veces los modelos sujetos a comparación
son de naturaleza fundamentalmente diferente
(por ejemplo, árboles de decisión frente a modelos lineales). En otras ocasiones, estamos comparando miembros de la misma clase de modelos que han sido entrenados con diferentes configuraciones de hiperparámetros.

Con los MLP, por ejemplo, es posible que deseemos comparar modelos con diferentes números de capas ocultas, diferentes números de unidades ocultas y varias opciones de funciones de activación aplicadas a cada capa oculta. Para determinar cuál es el mejor entre nuestros modelos candidatos, generalmente emplearemos un conjunto de datos de validación.

### Conjunto de datos de validación

En principio, no deberíamos tocar nuestro conjunto de prueba hasta que hayamos elegido todos nuestros hiperparámetros.
Si utilizáramos los datos de prueba en el proceso de selección del modelo, existe el riesgo de que podamos sobreajustar los datos de prueba. Entonces estaríamos en serios problemas. Si sobreajustamos nuestros datos de entrenamiento, siempre existe la evaluación de los datos de prueba para mantenernos honestos. Pero si sobreajustamos los datos de prueba, ¿cómo lo sabríamos?

Por lo tanto, nunca debemos confiar en los datos de prueba para la selección del modelo. Y, sin embargo, tampoco podemos confiar únicamente en los datos de entrenamiento para la selección del modelo porque no podemos estimar el error de generalización en los mismos datos que usamos para entrenar el modelo.


En aplicaciones prácticas, la imagen se vuelve más turbia. Si bien, idealmente, solo tocaríamos los datos de prueba una vez, para evaluar el mejor modelo o para comparar una pequeña cantidad de modelos entre sí, los datos de prueba del mundo real rara vez se descartan después de un solo uso. Rara vez podemos permitirnos un nuevo conjunto de prueba para cada ronda de experimentos.

La práctica común para abordar este problema
es dividir nuestros datos de tres maneras, incorporando un *conjunto de datos de validación* (o *conjunto de validación*) además de los conjuntos de datos de entrenamiento y prueba. 

![Imgur](https://i.imgur.com/jyEPbG9.png)

Un buen ejemplo para distinguir entre conjunto de prueba y de validación es lo que hace la plataforma Kaggle en sus competencias de aprendizaje automático. En sus inicios, Kaggle era solamente una plataforma de concursos donde las empresas publican problemas y los participantes compiten para construir el mejor algoritmo, generalmente con premios en efectivo. La organización d elos concursos consiste en:
1. el organizador debe separar su dataset en un conjunto de entrenamiento (que será publicado) y un conjunto de prueba (cuyas features serán publicadas, pero las etiquetas permanecerán ocultas). 
2. Los participantes podrán descargar los datos de entrenamiento y deberán elegir un modelo para presentar en la competencia. Para eso, deberán llevar adelante una selección de modelos generando un conjunto de validación a partir de los datos de entrenamiento.
3. Una vez seleccionado el modelo que mejor funcione con los datos de validación, se alimenta dicho modelo con las features del conjunto de prueba para obtener las etiquetas de prueba predichas por el modelo.
4. Se entregan las etiquetas de prueba predichas y el organizador las compara con las reales. El ganador es el modelo que menos erroes haya cometido. 
![Imgur](https://i.imgur.com/qA88YkJ.png)

De esta manera, los conjuntos de prueba y validación están bien diferenciados. El primero se usa para elegir el mejor modelo y el segundo se usa para evaluar el modelo elegido con datos que nunca vio en el entrenamiento.

A menos que se indique explícitamente lo contrario, en los experimentos de este curso en realidad estamos trabajando con lo que correctamente debería llamarse datos de entrenamiento y datos de validación, sin verdaderos conjuntos de prueba. Por lo tanto, reportado en cada experimento es realmente un accuracy de validación y no un verdadero accuracy del conjunto de pruebas.

### $K$*-fold cross-validation*

Cuando los datos de entrenamiento son escasos, es posible que ni siquiera podamos permitirnos mantener suficientes datos para constituir un conjunto de validación adecuado. Una solución popular a este problema es emplear $K$*-fold cross-validation*. Aquí, los datos de entrenamiento originales se dividen en $K$ subconjuntos que no se superponen. Luego, el entrenamiento y la validación del modelo se ejecutan $K$ veces, cada vez entrenando en $K-1$ subconjuntos y validando en un subconjunto diferente (el que no se usó para entrenar en esa ronda).
Finalmente, los errores de entrenamiento y validación se estiman promediando los resultados de los experimentos de $K$.

![Imgur](https://i.imgur.com/SpOFGyK.png)

In [1]:
import numpy as np
from sklearn.model_selection import KFold

import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader,ConcatDataset

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

from sklearn.model_selection import KFold

In [2]:
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def reset_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [4]:
def train(fold, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Fold/Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                fold,epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [5]:
def test(fold,model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set for fold {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        fold,test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [7]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

In [ ]:
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)

dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

In [10]:
model = MNISTNet().to(device)
model.apply(reset_weights)

MNISTNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [11]:
batch_size=32
folds=5
epochs=5

kfold=KFold(n_splits=folds,shuffle=True)

optimizer = optim.Adadelta(model.parameters())


In [14]:
dataset=ConcatDataset([dataset1,dataset2])

In [15]:
for train_idx,test_idx in kfold.split(dataset):
  print(test_idx)

[    1     9    10 ... 69991 69993 69996]
[    2     4     8 ... 69977 69990 69997]
[   11    24    26 ... 69989 69992 69994]
[    5     6    13 ... 69983 69986 69998]
[    0     3     7 ... 69984 69995 69999]


In [16]:
for fold,(train_idx,test_idx) in enumerate(kfold.split(dataset)):
  print('------------fold no---------{}----------------------'.format(fold))
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)

  trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=batch_size, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=batch_size, sampler=test_subsampler)

  model.apply(reset_weights)

  for epoch in range(1, epochs + 1):
    train(fold, model, device, trainloader, optimizer, epoch)
    test(fold,model, device, testloader)

------------fold no---------0----------------------
Train Fold/Epoch: 0/1 [0/70000 (0%)]	Loss: 2.329220
Train Fold/Epoch: 0/1 [16000/70000 (29%)]	Loss: 0.069491
Train Fold/Epoch: 0/1 [32000/70000 (57%)]	Loss: 0.174078
Train Fold/Epoch: 0/1 [48000/70000 (86%)]	Loss: 0.206182

Test set for fold 0: Average loss: 0.0132, Accuracy: 13720/70000 (20%)

Train Fold/Epoch: 0/2 [0/70000 (0%)]	Loss: 0.006167
Train Fold/Epoch: 0/2 [16000/70000 (29%)]	Loss: 0.008713
Train Fold/Epoch: 0/2 [32000/70000 (57%)]	Loss: 0.263678
Train Fold/Epoch: 0/2 [48000/70000 (86%)]	Loss: 0.008075

Test set for fold 0: Average loss: 0.0105, Accuracy: 13785/70000 (20%)

Train Fold/Epoch: 0/3 [0/70000 (0%)]	Loss: 0.002479
Train Fold/Epoch: 0/3 [16000/70000 (29%)]	Loss: 0.139068
Train Fold/Epoch: 0/3 [32000/70000 (57%)]	Loss: 0.025168
Train Fold/Epoch: 0/3 [48000/70000 (86%)]	Loss: 0.079616

Test set for fold 0: Average loss: 0.0106, Accuracy: 13807/70000 (20%)

Train Fold/Epoch: 0/4 [0/70000 (0%)]	Loss: 0.145906
Train Fo